In [6]:
import numpy as np
import pandas as pd
import pandas_profiling
from pandas.io.json import json_normalize 

import json
import logging

import psycopg2
from psycopg2.extras import Json

from sqlalchemy import create_engine, MetaData
import io

from input import json_schema

In [7]:
with open("data/source-fhir1.json") as f:
    d = json.load(f)

awhcuris = json_normalize(d)
data = awhcuris['entry'].loc[0]
data[:2]

[{'fullUrl': 'urn:uuid:315ae655-f8ac-496a-ae88-223b115c5696',
  'resource': {'resourceType': 'Basic',
   'id': '315ae655-f8ac-496a-ae88-223b115c5696',
   'meta': {'profile': ['http://standardhealthrecord.org/fhir/StructureDefinition/shr-entity-Person']},
   'code': {'coding': [{'system': 'http://standardhealthrecord.org/fhir/basic-resource-type',
      'code': 'shr-entity-Person',
      'display': 'shr-entity-Person'}]}}},
 {'fullUrl': 'urn:uuid:192546fa-8ab1-4563-875a-7a23f9cb5966',
  'resource': {'resourceType': 'Patient',
   'id': '192546fa-8ab1-4563-875a-7a23f9cb5966',
   'meta': {'profile': ['http://standardhealthrecord.org/fhir/StructureDefinition/shr-entity-Patient']},
   'text': {'status': 'generated',
    'div': '<div xmlns="http://www.w3.org/1999/xhtml">Generated by <a href="https://github.com/synthetichealth/synthea">Synthea</a>.Version identifier: v2.0.0-21-gee6136bb\n .   Person seed: -8290206020855395802  Population seed: 21</div>'},
   'extension': [{'url': 'http://hl7.o

In [8]:
json_normalize(data[:2]).T

,0,1
fullUrl,urn:uuid:315ae655-f8ac-496a-ae88-223b115c5696,urn:uuid:192546fa-8ab1-4563-875a-7a23f9cb5966
resource.address,NaN,[{'extension': [{'url': 'http://hl7.org/fhir/S...
resource.birthDate,NaN,2009-09-30
resource.code.coding,[{'system': 'http://standardhealthrecord.org/f...,NaN
resource.communication,NaN,[{'language': {'coding': [{'system': 'urn:ietf...
resource.extension,NaN,[{'url': 'http://hl7.org/fhir/us/core/Structur...
resource.gender,NaN,male
resource.id,315ae655-f8ac-496a-ae88-223b115c5696,192546fa-8ab1-4563-875a-7a23f9cb5966
resource.identifier,NaN,[{'system': 'https://github.com/synthetichealt...
resource.maritalStatus.coding,NaN,[{'system': 'http://hl7.org/fhir/v3/MaritalSta...


In [15]:

def flattenDict(d, result=None):
    if result is None:
        result = {}
    for key in d:
        value = d[key]
        #if isinstance(value, dict):
        if d['type'] =='object':
            value1 = {}
            for keyIn in value:
                value1[".".join([key,keyIn])]=value[keyIn]
            flattenDict(value1, result)
        elif isinstance(value, (list, tuple)):   
            for indexB, element in enumerate(value):
                if isinstance(element, dict):
                    value1 = {}
                    index = 0
                    for keyIn in element:
                        newkey = ".".join([key,keyIn])        
                        value1[".".join([key,keyIn])]=value[indexB][keyIn]
                        index += 1
                    for keyA in value1:
                        flattenDict(value1, result)   
        else:
            result[key]=value
    return result

#json_normalize(test_data).T

flat = flattenDict(data)
flat

TypeError: list indices must be integers or slices, not dict